In [22]:
list1 = [1, 2, 3, 4]
list2 = [3, 4]

items_only_in_list2 = list(set(list2) - set(list1))
# print(items_only_in_list2)  # Output: [5, 6]
if items_only_in_list2 is None or len(items_only_in_list2) == 0:
    print("empty or None")
else:
    print(items_only_in_list2)

empty or None


In [12]:
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
import numpy as np

def read_file(file_name, file_type='csv', encoding=None):
    if not encoding:
        encoding = 'utf-8'

    if file_type == 'csv':
        # Read the file with no header
        df = pd.read_csv(file_name, header=None, encoding=encoding)
    elif file_type == 'excel':
        # Read the file with no header
        df = pd.read_excel(file_name, header=None, encoding=encoding)
    elif file_type == 'txt':
        # Read the file with no header
        df = pd.read_csv(file_name, header=None, delimiter='\t', encoding=encoding)
    return df

def read_and_process_file(file_name, file_type='csv'):

    well_name = file_name.split('.')[0]
    file_type = file_name.split('.')[-1]

    try:
        df = read_file(file_name, file_type)
    except UnicodeDecodeError:
        df = read_file(file_name, file_type, 'latin-1')

    # Combine the first two rows to create the header
    header = df.iloc[0].str.strip() + '__' + df.iloc[1].str.strip()

    # Remove the first two rows
    df = df.iloc[2:]

    # Set the new header
    df.columns = header

    df["well"] = well_name

    # Print out the new column names
    print(df.columns)

    return df


def clean_df(df, columns):
    # Drop rows where all specified columns are 0
    for column in columns:
        df = df[df[column] != 0]

    # Convert specified columns to numeric and handle non-numeric values
    for column in columns:
        df[column] = pd.to_numeric(df[column], errors='coerce')

    # Replace -999.25 with NaN
    df = df.replace(-999.25, np.nan)

    # Drop rows where any specified column is NaN
    df = df.dropna(subset=columns)

    return df

def remove_outliers(df, columns, cleaned_df=False):

    if not cleaned_df:
        df = clean_df(df, columns)

    for column in columns:
        # Convert specified columns to numeric and handle non-numeric values
        df[column] = pd.to_numeric(df[column], errors='coerce')

        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        df = df[(df[column] >= Q1 - 1.5*IQR) &
                (df[column] <= Q3 + 1.5*IQR)]
        
    return df


def plot_3d_scatter(df, x_col, y_col, z_col, color_col=None, size_col=None, width=800, height=800, marker_size=3):
    # Create a copy of the dataframe and round the columns
    round_num = 2

    df_copy = df.copy()
    df_copy[x_col] = pd.to_numeric(df_copy[x_col], errors='coerce').round(round_num)
    df_copy[y_col] = pd.to_numeric(df_copy[y_col], errors='coerce').round(round_num)
    df_copy[z_col] = pd.to_numeric(df_copy[z_col], errors='coerce').round(round_num)

    fig = px.scatter_3d(df_copy, x=x_col, y=y_col, z=z_col,
                        color=color_col, size=size_col)
    fig.update_traces(marker=dict(size=marker_size))
    fig.update_layout(width=width, height=height)

    # Update the starting point of each axis to 0
    fig.update_scenes(xaxis=dict(range=[0, df_copy[x_col].max()*1.2]),
                      yaxis=dict(range=[0, df_copy[y_col].max()*1.2]),
                      zaxis=dict(range=[0, df_copy[z_col].max()*1.2]))
    
    fig.show()


def plot_2d_scatter(df, x_col, y_col, color_col=None, size_col=None, width=800, height=800, marker_size=5):
    # Create a copy of the dataframe and round the columns
    round_num = 2

    df_copy = df.copy()
    df_copy[x_col] = pd.to_numeric(
        df_copy[x_col], errors='coerce').round(round_num)
    df_copy[y_col] = pd.to_numeric(
        df_copy[y_col], errors='coerce').round(round_num)
    
    fig = px.scatter(df_copy, x=x_col, y=y_col, color=color_col, size=size_col)
    fig.update_traces(marker=dict(size=marker_size))
    fig.update_layout(width=width, height=height)

    # Update the starting point of each axis to 0
    fig.update_scenes(xaxis=dict(range=[0, df_copy[x_col].max()*1.2]),
                      yaxis=dict(range=[0, df_copy[y_col].max()*1.2]))

    fig.show()

In [13]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

def perform_kmeans(df, columns, k=None):
    # Extract the columns for clustering
    data = df[columns]

    if k is None:
        # List to hold the SSE for each k
        sse = []

        # Test k from 1 to 10
        for k in range(1, 11):
            kmeans = KMeans(n_clusters=k, random_state=1)
            kmeans.fit(data)
            sse.append(kmeans.inertia_)

        # Plot the SSE for each k
        plt.plot(range(1, 11), sse)
        plt.xticks(range(1, 11))
        plt.xlabel("Number of Clusters")
        plt.ylabel("SSE")
        plt.show()

        # Find the elbow point
        k = sse.index(min(sse)) + 1

    # Perform KMeans with the specified or optimal number of clusters
    kmeans = KMeans(n_clusters=k, random_state=1)
    kmeans.fit(data)

    # Add the cluster labels to the original DataFrame
    df['cluster'] = kmeans.labels_

    return df

In [14]:
df = read_and_process_file('./data_to_work/SELUANG.txt')
df = clean_df(df, ['ROPA_AVG__m/h', 'BITRPM_AVG__rpm', "TORQ_AVG__kLbf.ft", "WOB_AVG__kLbf"])
df = remove_outliers(
    df, ['ROPA_AVG__m/h', 'BITRPM_AVG__rpm', "TORQ_AVG__kLbf.ft", "WOB_AVG__kLbf"])
bit_dia = 12.25
df["MSE__ksi"] = df["WOB_AVG__kLbf"]/(3.14*(bit_dia/2)**2) + 480*df["TORQ_AVG__kLbf.ft"]*df["BITRPM_AVG__rpm"]/(bit_dia**2 * 3.28)
df["DOC__in/rev"] = df["ROPA_AVG__m/h"] * 39.37 / 60 / df["BITRPM_AVG__rpm"]
print(df.columns)

Index(['DEPREAMMD__m', 'DEPREAMTVD__m', 'ROPA_AVG__m/h', 'WOB_AVG__kLbf',
       'TORQ_AVG__kLbf.ft', 'SURFRPM_AVG__rpm', 'MOTORRPM_AVG__rpm',
       'BITRPM_AVG__rpm', 'SPP_AVG__Psi', 'CHKP_AVG__Psi', 'CEMENTP_AVG__Psi',
       'SPM01_AVG__Stk/min', 'SPM02_AVG__Stk/min', 'SPM03_AVG__Stk/min',
       'PITACTIVE_AVG__bbl', 'FLOWIN_AVG__gal/min', 'FLOWOUTP_AVG__%',
       'DIN_AVG__ppg', 'TIN_AVG__°C', 'DOUT_AVG__ppg', 'TOUT_AVG__°C', 'well'],
      dtype='object')
Index(['DEPREAMMD__m', 'DEPREAMTVD__m', 'ROPA_AVG__m/h', 'WOB_AVG__kLbf',
       'TORQ_AVG__kLbf.ft', 'SURFRPM_AVG__rpm', 'MOTORRPM_AVG__rpm',
       'BITRPM_AVG__rpm', 'SPP_AVG__Psi', 'CHKP_AVG__Psi', 'CEMENTP_AVG__Psi',
       'SPM01_AVG__Stk/min', 'SPM02_AVG__Stk/min', 'SPM03_AVG__Stk/min',
       'PITACTIVE_AVG__bbl', 'FLOWIN_AVG__gal/min', 'FLOWOUTP_AVG__%',
       'DIN_AVG__ppg', 'TIN_AVG__°C', 'DOUT_AVG__ppg', 'TOUT_AVG__°C', 'well',
       'MSE__ksi', 'DOC__in/rev'],
      dtype='object')


In [15]:
plot_3d_scatter(df, 'WOB_AVG__kLbf', 'TORQ_AVG__kLbf.ft', 'DOC__in/rev')

In [17]:
plot_3d_scatter(df, 'DOC__in/rev', 'TORQ_AVG__kLbf.ft', 'MSE__ksi')

In [16]:
# Read data_original
df = read_and_process_file("data_original.csv")

df = clean_df(df, ['ROPA_AVG__m/h', 'BITRPM_AVG__rpm',
              'DOC__in/rev', 'TORQ_AVG__kLbf.ft', 'WOB_AVG__tf'])
df = remove_outliers(df, 'DOC__in/rev')
df = remove_outliers(df, 'TORQ_AVG__kLbf.ft')

plot_2d_scatter(df, x_col='WOB_AVG__tf', y_col='ROPA_AVG__m/h', marker_size=7)

Index(['DEPMD__m', 'WOB_AVG__tf', 'DEPTVD__m', 'ROPA_AVG__m/h',
       'TORQ_AVG__kLbf.ft', 'SURFRPM_AVG__rpm', 'MOTORRPM_AVG__rpm',
       'BITRPM_AVG__rpm', 'SPP_AVG__Psi', 'CHKP_AVG__Psi', 'CEMENTP_AVG__Psi',
       'SPM01_AVG__Stk/min', 'SPM02_AVG__Stk/min', 'SPM03_AVG__Stk/min',
       'PITACTIVE_AVG__bbl', 'FLOWIN_AVG__gal/min', 'FLOWOUTP_AVG__%',
       'DIN_AVG__ppg', 'TIN_AVG__C', 'DOUT_AVG__ppg', 'TOUT_AVG__C', 'HP__hp',
       'DOC__in/rev', 'MSE__ksi', 'Comments__TOC/TOP'],
      dtype='object')


In [17]:
plot_3d_scatter(df, 'WOB_AVG__tf', 'TORQ_AVG__kLbf.ft',
                'DOC__in/rev')

In [18]:
df = perform_kmeans(df, ['WOB_AVG__tf', 'TORQ_AVG__kLbf.ft',
                         'DOC__in/rev'], k=2)
plot_3d_scatter(df, 'WOB_AVG__tf', 'TORQ_AVG__kLbf.ft',
                'DOC__in/rev', 'cluster')
df.to_csv('data_original_cluster.csv', index=None)
df.columns

c:\Users\phamsonn\Anaconda3\envs\funix_DL\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



Index(['DEPMD__m', 'WOB_AVG__tf', 'DEPTVD__m', 'ROPA_AVG__m/h',
       'TORQ_AVG__kLbf.ft', 'SURFRPM_AVG__rpm', 'MOTORRPM_AVG__rpm',
       'BITRPM_AVG__rpm', 'SPP_AVG__Psi', 'CHKP_AVG__Psi', 'CEMENTP_AVG__Psi',
       'SPM01_AVG__Stk/min', 'SPM02_AVG__Stk/min', 'SPM03_AVG__Stk/min',
       'PITACTIVE_AVG__bbl', 'FLOWIN_AVG__gal/min', 'FLOWOUTP_AVG__%',
       'DIN_AVG__ppg', 'TIN_AVG__C', 'DOUT_AVG__ppg', 'TOUT_AVG__C', 'HP__hp',
       'DOC__in/rev', 'MSE__ksi', 'Comments__TOC/TOP', 'cluster'],
      dtype='object')

In [8]:
plot_3d_scatter(df, 'DOC__in/rev', 'TORQ_AVG__kLbf.ft',
                'MSE__ksi', color_col='cluster')

In [20]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from scipy.optimize import minimize
import numpy as np
from tqdm import tqdm

(lower_bound_WOB, upper_bound_WOB) = (1, 30)   # Bounds for WOB (tf)
(lower_bound_RPM, upper_bound_RPM) = (20, 200)   # Bounds for RPM (rpm)
(lower_bound_Torque, upper_bound_Torque) = (
    1, 20)  # Bounds for Torque (kLbf-ft)
(lower_bound_ROPA_AVG, upper_bound_ROPA_AVG) = (
    1, 300)  # Bounds for ROPA_AVG (m/h)

# Load the data
# replace with your actual path
original_data = pd.read_csv('data_original_cluster.csv')

# Function to perform optimization
def perform_optimization(data):
    # Assuming that 'data' has the relevant features and the target variable
    # Replace 'target_variable' with the actual target column name
    X = data[['WOB_AVG__tf', 'BITRPM_AVG__rpm',
            'TORQ_AVG__kLbf.ft', 'ROPA_AVG__m/h']]
    y = data['MSE__ksi']

    # Train the random forest model
    rf_model = RandomForestRegressor()
    rf_model.fit(X, y)

    # Define the objective function for optimization


    def objective_function(params):
        # Unpack the parameters
        WOB, RPM, Torque, ROPA_AVG = params
        # Create a DataFrame with the parameters in the same order as the training data
        X_new = pd.DataFrame([params], columns=[
                            'WOB_AVG__tf', 'BITRPM_AVG__rpm', 'TORQ_AVG__kLbf.ft', 'ROPA_AVG__m/h'])
        # Negate the model's prediction to perform minimization
        return -rf_model.predict(X_new)[0]


    # Define the bounds for each parameter (replace with your actual bounds)
    bounds = [
        (lower_bound_WOB, upper_bound_WOB),   # Bounds for WOB
        (lower_bound_RPM, upper_bound_RPM),   # Bounds for RPM
        (lower_bound_Torque, upper_bound_Torque),  # Bounds for Torque
        (lower_bound_ROPA_AVG, upper_bound_ROPA_AVG)  # Bounds for ROPA_AVG
    ]

    # Define the initial guess for each parameter (replace with your actual initial guess or mean)
    initial_guess = [
        data['WOB_AVG__tf'].mean(),
        data['BITRPM_AVG__rpm'].mean(),
        data['TORQ_AVG__kLbf.ft'].mean(),
        data['ROPA_AVG__m/h'].mean()
    ]

    # Return the trained model and the result
    return rf_model

# Optimizing parameters
def monte_carlo_optimization(data, rf_model, iterations=500):
    # Define the bounds for each parameter
    bounds = [
        (lower_bound_WOB, upper_bound_WOB),   # Bounds for WOB
        (lower_bound_RPM, upper_bound_RPM),   # Bounds for RPM
        (lower_bound_Torque, upper_bound_Torque),  # Bounds for Torque
        (lower_bound_ROPA_AVG, upper_bound_ROPA_AVG)  # Bounds for ROPA_AVG
    ]

    # Define the objective function for optimization
    def objective_function(params):
        # Create a DataFrame with the parameters in the same order as the training data
        X_new = pd.DataFrame([params], columns=[
                             'WOB_AVG__tf', 'BITRPM_AVG__rpm', 'TORQ_AVG__kLbf.ft', 'ROPA_AVG__m/h'])
        # Negate the model's prediction to perform minimization
        return -rf_model.predict(X_new)[0]

    results = []
    
    for _ in tqdm(range(iterations)):  # wrap your loop with tqdm
    # for _ in range(iterations):
        # Random initial guess within bounds
        initial_guess = [
            np.random.uniform(lower_bound_WOB, upper_bound_WOB),
            np.random.uniform(lower_bound_RPM, upper_bound_RPM),
            np.random.uniform(lower_bound_Torque, upper_bound_Torque),
            np.random.uniform(lower_bound_ROPA_AVG, upper_bound_ROPA_AVG)
        ]
        # Perform the optimization using the provided model
        result = minimize(
            objective_function,
            initial_guess,
            method='L-BFGS-B',
            bounds=bounds
        )

        results.append((result.x, -result.fun))

    # Analyze the results to find ranges
    optimized_params = np.array([res[0] for res in results])
    mse_values = np.array([res[1] for res in results])

    # Determine the range of parameters for the lower MSE values
    low_mse_threshold = np.percentile(
        mse_values, 10)  # Adjust percentile as needed
    low_mse_indices = mse_values <= low_mse_threshold
    low_mse_params = optimized_params[low_mse_indices]

    return low_mse_params, mse_values[low_mse_indices]


def get_param_ranges(low_mse_params):
    param_ranges = {
        'WOB': (low_mse_params[:, 0].min(), low_mse_params[:, 0].max()),
        'RPM': (low_mse_params[:, 1].min(), low_mse_params[:, 1].max()),
        'Torque': (low_mse_params[:, 2].min(), low_mse_params[:, 2].max()),
        'ROPA_AVG': (low_mse_params[:, 3].min(), low_mse_params[:, 3].max()),
    }
    return param_ranges


def print_results(cluster, param_ranges, low_mses):
    print(f'Cluster: {cluster}')
    print(f'Parameter Ranges for Low MSE:')
    for param, (low, high) in param_ranges.items():
        print(f'{param}: {low:.2f} to {high:.2f}')
    print(
        f'Corresponding MSE Range: {low_mses.min():.2f} to {low_mses.max():.2f}')
    print()


# Then you can call these functions for each cluster
for cluster in original_data['cluster'].unique():
    clustered_data = original_data[original_data['cluster'] == cluster]
    rf_model = perform_optimization(clustered_data)
    low_mse_params, low_mses = monte_carlo_optimization(
        clustered_data, rf_model)
    param_ranges = get_param_ranges(low_mse_params)
    print_results(cluster, param_ranges, low_mses)

100%|██████████| 500/500 [00:27<00:00, 18.23it/s]


Cluster: 0
Parameter Ranges for Low MSE:
WOB: 1.04 to 29.44
RPM: 25.20 to 197.72
Torque: 1.04 to 15.85
ROPA_AVG: 113.10 to 297.23
Corresponding MSE Range: 8.44 to 8.50



100%|██████████| 500/500 [00:25<00:00, 19.40it/s]

Cluster: 1
Parameter Ranges for Low MSE:
WOB: 1.04 to 29.94
RPM: 20.66 to 199.81
Torque: 4.34 to 19.94
ROPA_AVG: 64.88 to 299.37
Corresponding MSE Range: 7.25 to 7.41



In [32]:
# Optimizing parameters
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from scipy.optimize import minimize
import numpy as np


def monte_carlo_optimization(data, iterations=100):
    results = []
    for _ in range(iterations):
        # Random initial guess within bounds
        initial_guess = [
            np.random.uniform(lower_bound_WOB, upper_bound_WOB),
            np.random.uniform(lower_bound_RPM, upper_bound_RPM),
            np.random.uniform(lower_bound_Torque, upper_bound_Torque),
            np.random.uniform(lower_bound_ROPA_AVG, upper_bound_ROPA_AVG)
        ]
        result = minimize(
            objective_function,
            initial_guess,
            method='L-BFGS-B',
            bounds=bounds
        )
        results.append((result.x, -result.fun))

    # Analyze the results to find ranges
    optimized_params = np.array([res[0] for res in results])
    mse_values = np.array([res[1] for res in results])

    # Determine the range of parameters for the lower MSE values
    low_mse_indices = mse_values <= np.percentile(
        mse_values, 10)  # Adjust percentile as needed
    low_mse_params = optimized_params[low_mse_indices]

    return low_mse_params, mse_values[low_mse_indices]


# Then you can call this function for each cluster
for cluster in original_data['cluster'].unique():
    clustered_data = original_data[original_data['cluster'] == cluster]
    low_mse_params, low_mses = monte_carlo_optimization(clustered_data)

    # Process the low_mse_params to find the range of parameters
    param_ranges = {
        'WOB': (low_mse_params[:, 0].min(), low_mse_params[:, 0].max()),
        'RPM': (low_mse_params[:, 1].min(), low_mse_params[:, 1].max()),
        'Torque': (low_mse_params[:, 2].min(), low_mse_params[:, 2].max()),
        'ROPA_AVG': (low_mse_params[:, 3].min(), low_mse_params[:, 3].max()),
    }

    print(f'Cluster: {cluster}')
    print(f'Parameter Ranges for Low MSE:')
    for param, (low, high) in param_ranges.items():
        print(f'{param}: {low:.2f} to {high:.2f}')
    print(
        f'Corresponding MSE Range: {low_mses.min():.2f} to {low_mses.max():.2f}')
    print()

NameError: name 'objective_function' is not defined

### Normalize data

In [49]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Load the dataset
data_path = 'data_original_cluster.csv'
original_data = pd.read_csv(data_path)

# Select features for normalization
# Assuming that these are the features in the dataset that we want to normalize
# (based on the provided code snippet)
features_to_normalize = ['WOB_AVG__tf',
                         'BITRPM_AVG__rpm', 'TORQ_AVG__kLbf.ft', 'ROPA_AVG__m/h']

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler to the features and transform
original_data[features_to_normalize] = scaler.fit_transform(
    original_data[features_to_normalize])

# Check the first few rows of the dataset to confirm normalization
normalized_data_head = original_data.head()

normalized_data_head

,DEPMD__m,WOB_AVG__tf,DEPTVD__m,ROPA_AVG__m/h,TORQ_AVG__kLbf.ft,SURFRPM_AVG__rpm,MOTORRPM_AVG__rpm,BITRPM_AVG__rpm,SPP_AVG__Psi,CHKP_AVG__Psi,...,FLOWOUTP_AVG__%,DIN_AVG__ppg,TIN_AVG__C,DOUT_AVG__ppg,TOUT_AVG__C,HP__hp,DOC__in/rev,MSE__ksi,Comments__TOC/TOP,cluster
0,2671,5.428037,2536.87,-1.244065,-2.344917,48,0,-5.385028,676,0,...,24.95,11.69,38.025,11.69,34.7,41.492765,0.748852,8.20,TOC,0
1,2687,0.191640,2550.40,-0.536158,-1.805236,48,0,-5.385028,862,0,...,24.58,11.69,40.323,11.69,34.4,43.594821,0.887741,7.19,TOC,1
2,2688,0.169545,2551.26,-0.580750,-1.875630,48,0,-5.385028,865,0,...,23.81,11.69,40.254,11.70,34.5,43.320640,0.878992,7.22,TOC,1
3,2689,0.169545,2552.10,-0.456031,-1.899094,48,0,-5.385028,865,0,...,25.21,11.69,40.191,11.70,34.5,43.229246,0.903461,7.01,TOC,1
4,2690,0.169545,2552.95,-0.428160,-1.946023,48,0,-5.385028,866,0,...,24.93,11.69,40.136,11.70,34.6,43.046458,0.908929,6.94,TOC,1


In [50]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

# Selecting the target variable based on the provided code snippet
target_variable = 'MSE__ksi'

# Splitting the dataset into features and target variable
X = original_data[features_to_normalize]
y = original_data[target_variable]

# Splitting the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Initializing models
linear_model = LinearRegression()
ridge_model = Ridge(random_state=42)
knn_model = KNeighborsRegressor()
svm_model = SVR()

# Dictionary to store models and their names
models = {
    'Linear Regression': linear_model,
    'Ridge Regression': ridge_model,
    'K-Nearest Neighbors': knn_model,
    'Support Vector Machine': svm_model
}

# Dictionary to store results
results = {}

# Training and evaluating each model
for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    # Predict on the test set
    y_pred = model.predict(X_test)
    # Calculate MSE
    mse = mean_squared_error(y_test, y_pred)
    # Store results
    results[model_name] = mse

results

{'Linear Regression': 0.32707822340174336,
 'Ridge Regression': 0.3261305804717096,
 'K-Nearest Neighbors': 0.10426977083333326,
 'Support Vector Machine': 0.09170705939913572}

In [52]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Load your dataset
data = pd.read_csv('data_original_cluster.csv')

# Assuming your dataset is already loaded and is named 'data'
# And assuming 'cluster' is the column with cluster identifiers

# Define the bounds for your parameters (example bounds given)
bounds = [(1, 20), (20, 200), (1, 20), (1, 300)]

# Normalize the features
scaler = StandardScaler()
features = ['WOB_AVG__tf', 'BITRPM_AVG__rpm',
            'TORQ_AVG__kLbf.ft', 'ROPA_AVG__m/h']
data[features] = scaler.fit_transform(data[features])

# Define the model architecture


def build_model(input_shape):
    model = Sequential([
        Dense(64, input_dim=input_shape, activation='relu'),
        Dropout(0.1),
        Dense(32, activation='relu'),
        Dropout(0.1),
        Dense(16, activation='relu'),
        Dense(1, activation='linear')  # Output layer for regression
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='mean_squared_error')
    return model

# Function to optimize the MSE for a given cluster


def optimize_cluster(cluster_data, bounds):
    # Splitting the data
    X = cluster_data[features]
    y = cluster_data['MSE__ksi']

    # Further split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)

    # Build and train the model
    model = build_model(X_train.shape[1])
    model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=0)

    # Define the objective function for the optimizer to minimize
    def objective(params):
        params = np.array(params).reshape(1, -1)
        params = scaler.transform(params)  # Make sure to scale the parameters
        prediction = model.predict(params)
        return prediction[0][0]

    # Perform optimization
    result = minimize(objective, x0=X.mean().tolist(),
                      bounds=bounds, method='L-BFGS-B')

    # Scale back the optimized parameters to their original scale
    optimized_params = scaler.inverse_transform(
        np.array(result.x).reshape(1, -1))

    # Evaluate the model with optimized parameters
    min_mse = objective(result.x)

    return optimized_params.flatten().tolist(), min_mse


# Dictionary to hold optimization results for each cluster
optimization_results = {}

# Loop over each cluster to optimize
for cluster in data['cluster'].unique():
    cluster_data = data[data['cluster'] == cluster]
    params, min_mse = optimize_cluster(cluster_data, bounds)
    optimization_results[cluster] = {'Parameters': params, 'Min MSE': min_mse}

# Print the optimization results for each cluster
for cluster, results in optimization_results.items():
    print(
        f"Cluster: {cluster}, Optimized Parameters: {results['Parameters']}, Min MSE: {results['Min MSE']}")

c:\Users\phamsonn\Anaconda3\envs\funix_DL\lib\site-packages\sklearn\base.py:465: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



1/1 [==============================] - 0s 30ms/step


c:\Users\phamsonn\Anaconda3\envs\funix_DL\lib\site-packages\sklearn\base.py:465: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

c:\Users\phamsonn\Anaconda3\envs\funix_DL\lib\site-packages\sklearn\base.py:465: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

c:\Users\phamsonn\Anaconda3\envs\funix_DL\lib\site-packages\sklearn\base.py:465: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



1/1 [==============================] - 0s 29ms/step


c:\Users\phamsonn\Anaconda3\envs\funix_DL\lib\site-packages\sklearn\base.py:465: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

c:\Users\phamsonn\Anaconda3\envs\funix_DL\lib\site-packages\sklearn\base.py:465: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



...

1/1 [==============================] - 0s 28ms/step


c:\Users\phamsonn\Anaconda3\envs\funix_DL\lib\site-packages\sklearn\base.py:465: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

c:\Users\phamsonn\Anaconda3\envs\funix_DL\lib\site-packages\sklearn\base.py:465: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



1/1 [==============================] - 0s 34ms/step


c:\Users\phamsonn\Anaconda3\envs\funix_DL\lib\site-packages\sklearn\base.py:465: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

c:\Users\phamsonn\Anaconda3\envs\funix_DL\lib\site-packages\sklearn\base.py:465: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



Cluster: 0, Optimized Parameters: [1.675864886804223, 175.07635204481466, 5.96552913122378, 86.98717660902193], Min MSE: 16.23601722717285
Cluster: 1, Optimized Parameters: [1.675864886804223, 175.07635204481466, 5.96552913122378, 86.98717660902193], Min MSE: 23.108173370361328


In [46]:
# !pip install scikit-optimize
from math import pi
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args

# Conversion factors
uk_ton_to_lbm = 2240
m_to_ft = 3.28084

# Function to convert units


def convert(value, from_unit, to_unit):
    if from_unit == "uk_ton" and to_unit == "lbm":
        return value * uk_ton_to_lbm
    elif from_unit == "m" and to_unit == "ft":
        return value * m_to_ft
    else:
        raise ValueError(
            "Conversion from {} to {} not implemented.".format(from_unit, to_unit))

# Define the space of hyperparameters to optimize
space = [Real(lower_bound_WOB, upper_bound_WOB, name='WOB'),
         Real(lower_bound_RPM, upper_bound_RPM, name='RPM'),
         Real(lower_bound_Torque, upper_bound_Torque, name='Torque'),
         Real(lower_bound_ROPA_AVG, upper_bound_ROPA_AVG, name='ROPA_AVG')]

# Define the objective function
@use_named_args(space)
def objective(WOB, RPM, Torque, ROPA_AVG):
    # Convert WOB from UK tons to lbm and ROP from m to ft
    WOB_lbm = convert(WOB, "uk_ton", "lbm")
    ROP_ft = convert(ROPA_AVG, "m", "ft")

    # Calculate MSE in ksi
    MSE = WOB_lbm / (pi * 4.25 * 4.25 * 1000) + 480 * \
        Torque * RPM / (8.5 * 8.5 * ROP_ft)
    return MSE


# Run the optimization
res_gp = gp_minimize(objective, space, n_calls=100, random_state=0)

# Print the found minimum
print("Best MSE=%.4f" % res_gp.fun)
print("Best parameters:")
for param, val in zip(['WOB', 'RPM', 'Torque', 'ROPA_AVG'], res_gp.x):
    print(f"{param}: {val:.4f}")

c:\Users\phamsonn\Anaconda3\envs\funix_DL\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning:

The objective has been evaluated at this point before.

c:\Users\phamsonn\Anaconda3\envs\funix_DL\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning:

The objective has been evaluated at this point before.



Best MSE=0.1745
Best parameters:
WOB: 1.0000
RPM: 20.0000
Torque: 1.0000
ROPA_AVG: 300.0000


In [120]:
# import data_wrangling.xlsx as df
df = pd.read_excel('data_wrangling.xlsx')

# print out column names in df
print(df.columns)

df = clean_df(df, ['ROPA_AVG__m/h', 'BITRPM_AVG__rpm'])
df = remove_outliers(df, 'DOC__in/rev')
df = remove_outliers(df, 'TORQ_AVG__kLbf.ft')
plot_3d_scatter(df, 'WOB_AVG__tf ', 'TORQ_AVG__kLbf.ft',
                'DOC__in/rev', 'Comments__TOC/TOP')
plot_3d_scatter(df, 'DOC__in/rev', 'TORQ_AVG__kLbf.ft',
                'MSE__psi', 'DEPMD__m')

Index(['DEPMD__m', 'WOB_AVG__tf ', 'DEPTVD__m', 'ROPA_AVG__m/h',
       'TORQ_AVG__kLbf.ft', 'SURFRPM_AVG__rpm', 'MOTORRPM_AVG__rpm',
       'BITRPM_AVG__rpm', 'SPP_AVG__Psi', 'CHKP_AVG__Psi', 'CEMENTP_AVG__Psi',
       'SPM01_AVG__Stk/min', 'SPM02_AVG__Stk/min', 'SPM03_AVG__Stk/min',
       'PITACTIVE_AVG__bbl', 'FLOWIN_AVG__gal/min', 'FLOWOUTP_AVG__%',
       'DIN_AVG__ppg', 'TIN_AVG__C', 'DOUT_AVG__ppg', 'TOUT_AVG__C', 'HP__hp',
       'MSE__psi', 'Comments__TOC/TOP', 'DOC__in/rev'],
      dtype='object')


In [124]:
df = perform_kmeans(df, ['WOB_AVG__tf ', 'TORQ_AVG__kLbf.ft',
                         'DOC__in/rev'], k=2)
plot_3d_scatter(df, 'WOB_AVG__tf ', 'TORQ_AVG__kLbf.ft',
                'DOC__in/rev', 'cluster')

c:\Users\phamsonn\Anaconda3\envs\funix_DL\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [10]:
df['MSE__ksi'] = pd.to_numeric(df['MSE__ksi'], errors='coerce')
df['MSE__ksi'] = df['MSE__ksi'].round(2)
plot_3d_scatter(df, 'DOC__in/rev', 'TORQ_AVG__kLbf.ft',
                'MSE__ksi', 'cluster')

In [82]:
# Read the file with no header
df = pd.read_csv('drilling_ucs.csv', header=None)

# Combine the first two rows to create the header
header = df.iloc[0].str.strip() + '__' + df.iloc[1].str.strip()

# Remove the first two rows
df = df.iloc[2:]

# Set the new header
df.columns = header

# Print out the new column names
print(df.columns)

Index(['Depth__m', 'DCAV__in', 'UCS-CA__psi', 'UCS-BD__psi', 'GR__gapi',
       'P40H__ohm.m', 'WOBA__klbm', 'RPMS__1/min', 'TQA__klbf.ft', 'ROPA__m/h',
       'DOC__in/rev', 'MSE__ksi'],
      dtype='object')


In [90]:
df = read_and_process_file('drilling_ucs.csv')
df = clean_df(df, ['ROPA__m/h', 'RPMS__1/min',
              'DOC__in/rev', 'TQA__klbf.ft', 'WOBA__klbm', 'Depth__m', 'MSE__ksi', 'UCS-BD__psi'])
df = clean_df(df, ['ROPA__m/h', 'RPMS__1/min',
              'DOC__in/rev', 'TQA__klbf.ft', 'WOBA__klbm', 'UCS-BD__psi'])
df = remove_outliers(df, 'DOC__in/rev')
df = remove_outliers(df, 'TQA__klbf.ft')
plot_3d_scatter(df, 'DOC__in/rev', 'TQA__klbf.ft',
                'MSE__ksi', 'UCS-BD__psi', 'WOBA__klbm')

In [87]:
plot_2d_scatter(df, 'DOC__in/rev', 
                'MSE__ksi', 'Depth__m', 'WOBA__klbm')

In [114]:
df = read_and_process_file('hard_stringers.csv')
df = clean_df(df, ['ROP5__m/h', 'RPM__c/min',
              'DOC__in/rev', 'STOR__kN.m', 'SWOB__1000', 'Depth__m', 'MSE__ksi', 'P40H#1__ohm.m'])
df = remove_outliers(df, 'DOC__in/rev')
df = remove_outliers(df, 'STOR__kN.m')
df = remove_outliers(df, 'SWOB__1000')
df = remove_outliers(df, 'P40H#1__ohm.m')
df = df[df['SWOB__1000']>0]
plot_3d_scatter(df, x_col='DOC__in/rev', y_col='STOR__kN.m',
                z_col='MSE__ksi', color_col='Depth__m', marker_size=3)
plot_3d_scatter(df, x_col='SWOB__1000', y_col='STOR__kN.m',
                z_col='DOC__in/rev', color_col='P40H#1__ohm.m', marker_size=3)

Index(['Depth__m', 'ROP5__m/h', 'STOR__kN.m', 'SWOB__1000', 'RPM__c/min',
       'CRPM__c/min', 'STICK__c/min', 'SPPA__psi', 'TFLO__gal/min',
       'DHAP__psi', 'VIB_X__gn', 'VIB_LAT__gn', 'SHKRSK__', 'SHKL__',
       'P40H#1__ohm.m', 'GR#1__gAPI', 'MSE__ksi', 'DOC__in/rev'],
      dtype='object')


## Parking Slot

In [ ]:
def plot_3d_scatter(df, x_col, y_col, z_col, color_col=None, size_col=None, width=800, height=800, marker_size=3, num_ticks=5):
    color = df[color_col] if color_col else None

    fig = go.Figure(data=[go.Scatter3d(
        x=df[x_col],
        y=df[y_col],
        z=df[z_col],
        mode='markers',
        marker=dict(
            size=marker_size,
            color=color,
            colorscale='Viridis',
            opacity=0.8
        )
    )])

    # Calculate dtick values for each axis
    x_dtick = (df[x_col].max() - df[x_col].min()) / num_ticks
    y_dtick = (df[y_col].max() - df[y_col].min()) / num_ticks
    z_dtick = (df[z_col].max() - df[z_col].min()) / num_ticks

    fig.update_layout(scene=dict(
        xaxis_title=x_col,
        yaxis_title=y_col,
        zaxis_title=z_col,
        xaxis=dict(
            tickmode='linear',
            tick0=0,
            dtick=x_dtick,
            tickformat=".2f",
            range=[0, df[x_col].max()]),  # Set min value to 0
        yaxis=dict(
            tickmode='linear',
            tick0=0,
            dtick=y_dtick,
            tickformat=".2f",
            range=[0, df[y_col].max()]),  # Set min value to 0
        zaxis=dict(
            tickmode='linear',
            tick0=0,
            dtick=z_dtick,
            tickformat=".2f",
            range=[0, df[z_col].max()]),  # Set min value to 0
    ),
        width=700,
        margin=dict(r=20, b=10, l=10, t=10))

    fig.show()